In [20]:
import torch

import meshzoo

from torchfem import Solid
from torchfem.io import export_mesh

# Material model

Isotropic linear elastic stiffness tensor

In [21]:
# Material model
E = 1000.0
nu = 0.3
lbd = (E * nu) / ((1.0 + nu) * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))

# E_xx, E_yy, E_zz, E_yz, E_xz, E_xy
C = torch.tensor(
    [
        [lbd + 2.0 * mu, lbd, lbd, 0.0, 0.0, 0],
        [lbd, lbd + 2.0 * mu, lbd, 0.0, 0.0, 0],
        [lbd, lbd, lbd + 2.0 * mu, 0.0, 0.0, 0],
        [0.0, 0.0, 0.0, mu, 0.0, 0],
        [0.0, 0.0, 0.0, 0.0, mu, 0],
        [0.0, 0.0, 0.0, 0.0, 0.0, mu],
    ]
)

# 3D Cube with hexaeders

In [22]:
# Generate cube
points, cells = meshzoo.cube_hexa(
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
)
nodes = torch.tensor(points)
elements = cells.tolist()

# Assign boundary conditions
forces = torch.zeros_like(nodes)
forces[nodes[:, 0] == 1.0, 0] = 100.0
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True

box = Solid(nodes, elements, forces, constraints, C)

u, f = box.solve()

export_mesh(box, "cube_hexa.vtk", nodal_data={"Disp": u})

# 3D Cube with tetrahedrons

In [23]:
# Generate cube
points, cells = meshzoo.cube_tetra(
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
)
nodes = torch.tensor(points)
elements = cells.tolist()

# Assign boundary conditions
forces = torch.zeros_like(nodes)
forces[nodes[:, 0] == 1.0, 0] = 100.0
constraints = torch.zeros_like(nodes, dtype=bool)
constraints[nodes[:, 0] == 0.0, :] = True

box = Solid(nodes, elements, forces, constraints, C)

u, f = box.solve()

export_mesh(box, "cube_tetra.vtk", nodal_data={"Disp": u})